## Load libraries

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import collections
import random

embedding_dict = np.load('embedding_dict.npy').item()
embedding_dim = 300

In [2]:
fable_text = """
long ago , the mice had a general council to consider what measures
they could take to outwit their common enemy , the cat . some said
this , and some said that but at last a young mouse got up and said
he had a proposal to make , which he thought would meet the case . 
you will all agree , said he , that our chief danger consists in the
sly and treacherous manner in which the enemy approaches us . now , 
if we could receive some signal of her approach , we could easily
escape from her . i venture , therefore , to propose that a small
bell be procured , and attached by a ribbon round the neck of the cat
. by this means we should always know when she was about , and could
easily retire while she was in the neighbourhood . this proposal met
with general applause , until an old mouse got up and said that is
all very well , but who is to bell the cat ? the mice looked at one
another and nobody spoke . then the old mouse said it is easy to
propose impossible remedies .
"""
# Replace the carriage return with space.
fable_text = fable_text.replace('\n','')

#this function puts all the words in a single column vector within a numpy array

def read_data(raw_text):
 content = raw_text
 content = content.split() #splits the text by spaces (default split character)
 content = np.array(content)
 content = np.reshape(content, [-1, ])
 return content

training_data = read_data(fable_text)

#Create dictionary and reverse dictionary with word ids

def build_dictionaries(words):
    count = collections.Counter(words).most_common() #creates list of word/count pairs;
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary) #len(dictionary) increases each iteration
        reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dictionaries(training_data)


#Create embedding array

doc_vocab_size = len(dictionary)


embeddings_tmp=[]

for i in range(doc_vocab_size):
    item = reverse_dictionary[i]
    if item in embedding_dict.keys():
        embeddings_tmp.append(tf.constant(embedding_dict[item],dtype=tf.float32))
        #embeddings_tmp.append(tf.Variable(embedding_dict[item],trainable=True,dtype=tf.float32))
        #rand_num = np.random.uniform(low=-0.2, high=0.2,size=embedding_dim)
        #embeddings_tmp.append(tf.Variable(rand_num,trainable=True,dtype=tf.float32))
    else:
        rand_num = np.random.uniform(low=-0.2, high=0.2,size=embedding_dim)
        embeddings_tmp.append(tf.Variable(rand_num,trainable=True,dtype=tf.float32))

# final embedding array corresponds to dictionary of words in the document
W0 = tf.convert_to_tensor(embeddings_tmp)


In [3]:
# model parameters
learning_rate = 0.001
n_input = 3 # this is the number of words that are read at a time
n_hidden_1 = 256
n_hidden_2 = 256

## Build graph

### Use `tf.data.Dataset` to build pipeline
`batch_size`: Scalar placeholder to accept mini-batch size.

`data_vector`: 1D placeholder to accept a vector of integers representing series of training words.

In [4]:
batch_size =  tf.placeholder(tf.int64,[])
data_vector = tf.placeholder(tf.int32,[None,])
data_size = tf.size(data_vector,out_type=tf.int64)

dataset = tf.data.Dataset.from_tensor_slices(
                                (tf.stack([data_vector[0:data_size-3],
                                           data_vector[1:data_size-2],
                                           data_vector[2:data_size-1]],
                                          axis=1),
                                   tf.reshape(data_vector[3:data_size],[-1,1]))
                                    )

dataset = dataset.shuffle(data_size).repeat().batch(batch_size)
iterator = dataset.make_initializable_iterator()
current_batch = iterator.get_next()

### Main part of compute graph.

`W`: keeps vocabulary embedding vectors.

`x`: a variable to store a batch of input words (represented by integers). During inference, `x` is used as a feedable object to receive a vector of integers representing input words.

`pred`: Predicted embedding vector(s) for batch of next word(s).


In [5]:
x = tf.Variable(0, [None, n_input])
weights = tf.Variable(tf.random_normal([n_hidden_2, embedding_dim]))
biases =  tf.Variable(tf.random_normal([embedding_dim]))

with tf.name_scope("embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[doc_vocab_size, embedding_dim]), trainable=False)
  
    x = current_batch[0]
    embedded_chars = tf.nn.embedding_lookup(W0,x)
    yy = tf.nn.embedding_lookup(W0,current_batch[1])
    y = tf.reshape(yy,shape=[-1,embedding_dim])

 
# reshape input data
x_unstack = tf.unstack(embedded_chars, n_input, 1)

# create RNN cells
rnn_cell = rnn.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(n_hidden_1,name='basic_lstm_cell'),
                             tf.nn.rnn_cell.LSTMCell(n_hidden_2,name='basic_lstm_cell')])

outputs, states = rnn.static_rnn(rnn_cell, x_unstack, dtype=tf.float32)

# capture only the last output
pred = tf.matmul(outputs[-1], weights) + biases

# Create loss function and optimizer
#cost = tf.reduce_mean(tf.nn.l2_loss(pred-y))
cost = tf.nn.l2_loss(pred-y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
save_W = W.assign(W0)
init=tf.global_variables_initializer()

##  Training (Run the graph)

In [6]:
# Map words in training data set to internal representation of integers.
training_data_int = np.asarray([dictionary[a] for a in list(training_data)],dtype=np.int32)


# Run the graph
step=0

acc_total = 0
loss_total = 0
training_iters =500
BATCH_SIZE = 16

sess = tf.Session()
sess.run(init)
sess.run([iterator.initializer] ,
         feed_dict={batch_size:BATCH_SIZE,
                    data_vector:training_data_int})
sess.run(W0)

while step < training_iters:
    _,loss= sess.run([optimizer, cost])
    if (step % 50 ==0): print("Loss:{}",loss)
    step +=1
sess.run(save_W)
print("Finished Optimization")

Loss:{} 2968.834
Loss:{} 135.11447
Loss:{} 134.31493
Loss:{} 165.87051
Loss:{} 102.77092
Loss:{} 110.6465
Loss:{} 90.925064
Loss:{} 77.81599
Loss:{} 92.27479
Loss:{} 60.00712
Finished Optimization


## Save checkpoints (ex: graph & variables)

In [7]:
tf.add_to_collection('ops',x)
tf.add_to_collection('ops',W)
tf.add_to_collection('ops',pred)

np.save('reverse_dictionary.npy', reverse_dictionary) 
np.save('dictionary.npy', dictionary) 

saver = tf.train.Saver()
saver.save(sess, './RNN_model',global_step=step)

sess.close()

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
[None, 3] has type list, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
[None, 3] has type list, but expected one of: int, long, bool
